In [1]:
import os
import pandas as pd
import glob
import stagger
import librosa
import seaborn as sns
import numpy as np

from matplotlib import pyplot as plt
from librosa import display
from librosa import feature
from stagger import read_tag


In [2]:
#wykrycie ścieżki, w której przechowuje pliki i program
working_directory = os.getcwd()
path = working_directory + '/wrapped/big data projekt/*.mp3' 
songs = glob.glob(path) 


In [3]:
"osobna funkcja do liczenia sredniej z macierzy"
def Average(lst):
    return sum(lst) / len(lst)


In [4]:

"""stworzenie pustych list na 20 wskaznikow mfcc"""
mfcc_0, mfcc_1, mfcc_2, mfcc_3, mfcc_4, mfcc_5, mfcc_6, mfcc_7, mfcc_8, mfcc_9, mfcc_10, mfcc_11, mfcc_12, mfcc_13, mfcc_14, mfcc_15, mfcc_16, mfcc_17, mfcc_18, mfcc_19 = [],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
mfcc_lists = [mfcc_0 , mfcc_1 ,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19]


In [5]:

"""stworzenie listy cech jednowymiarowych"""
feature1_list_i = [
 feature.chroma_stft,
 feature.spectral_centroid,
 feature.spectral_bandwidth,
 feature.spectral_rolloff ]
 
"""stworzenie listy cech dwuwymiarowych"""
feature2_list_ii = [
 feature.rms,
 feature.zero_crossing_rate ]

"""stworzenie listy cech mfcc"""
feature3_list_mfcc = [
    feature.mfcc ]


In [6]:


"""stworzenie funkcji obliczającej średnią z danej cechy i zwracającej wyniki do wektora feature_vector"""
def get_feature_vector(y,sr,mp3): 
    feat_artist = [mp3.artist]
    feat_vect_i = [ np.mean(funct(y=y,sr=sr)) for funct in feature1_list_i]
    feat_vect_ii = [ np.mean(funct(y=y)) for funct in feature2_list_ii] 
    mfcc = [np.mean(funct(y=y,sr=sr).T, axis = 0) for funct in feature3_list_mfcc]
    feat_vect_iii = list(mfcc[0])
    
    #feature_vector jako lista przechowująca wszystkie cechy
    feature_vector = feat_artist + feat_vect_i + feat_vect_ii + feat_vect_iii
    return feature_vector


In [7]:
audio_features = []
licznik = 0
for song in songs:
    mp3 = stagger.read_tag(song)
    print(f'dodaje piosenke: {mp3.title}')
    dl = librosa.get_duration(path=song)
    #odebranie sygnalu dla calej piosenki
    y , sr = librosa.load(song,sr = 22858)
    
    #wyelminowanie tła i zostawienie wokalu
    S_full, phase = librosa.magphase(librosa.stft(y=y))
    S_filter = librosa.decompose.nn_filter(S_full,
                                       aggregate=np.median,
                                       metric='cosine',
                                       width=int(librosa.time_to_frames(2, sr=sr)))
    #stworzenie filtra
    S_filter = np.minimum(S_full, S_filter)
    margin_i, margin_v = 2, 10
    power = 2

    mask_i = librosa.util.softmask(S_filter,
                               margin_i * (S_full - S_filter),
                               power=power)

    mask_v = librosa.util.softmask(S_full - S_filter,
                               margin_v * S_filter,
                               power=power)
    S_foreground = mask_v * S_full
    S_background = mask_i * S_full
    y_foreground = librosa.istft(S_foreground * phase)

    S_foreground = mask_v * S_full
    S_background = mask_i * S_full
    
    feature_vector = get_feature_vector(y_foreground, sr,mp3)
    audio_features.append(feature_vector)
    licznik+=1
    

dodaje piosenke: (Nice Dream)
dodaje piosenke: 10:15 Saturday Night
dodaje piosenke: 15 Step
dodaje piosenke: 2 + 2 = 5
dodaje piosenke: A Forest
dodaje piosenke: A Punch Up at a Wedding
dodaje piosenke: A Short Term Effect
dodaje piosenke: A Wolf At the Door
dodaje piosenke: Airbag
dodaje piosenke: All I Need
dodaje piosenke: Always in the Sun
dodaje piosenke: Amour dans le motu
dodaje piosenke: Antitaxi
dodaje piosenke: Anyone Can Play Guitar
dodaje piosenke: Ashes to Ashes - 2017 Remaster
dodaje piosenke: Backdrifts
dodaje piosenke: bitter cold
dodaje piosenke: Black Star
dodaje piosenke: Blackout - 2017 Remaster
dodaje piosenke: Blackstar
dodaje piosenke: Bloom
dodaje piosenke: Blow Out
dodaje piosenke: Bodysnatchers
dodaje piosenke: Bones
dodaje piosenke: Boys Don't Cry
dodaje piosenke: Bullet Proof ... I Wish I Was
dodaje piosenke: Burn the Witch
dodaje piosenke: Changes - 2015 Remaster
dodaje piosenke: China Girl - 2018 Remaster
dodaje piosenke: Climbing Up the Walls
dodaje pios

In [10]:
header =['artist','chroma_stft','spectral_centroid','spectral_bandwidth','spectral_rolloff',
    'rmse','zero_crossing_rate','mfcc_0','mfcc_1','mfcc_2','mfcc_3','mfcc_4','mfcc_5',
    'mfcc_6','mfcc_7','mfcc_8','mfcc_9','mfcc_10','mfcc_11','mfcc_12','mfcc_13','mfcc_14',
    'mfcc_15','mfcc_16','mfcc_17','mfcc_18','mfcc_19']

In [11]:
df = pd.DataFrame(audio_features,columns = header)

In [12]:
writer = pd.ExcelWriter('wrapped_dataset_vocal_270.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='welcome', index=False)
writer.save()